This try to install the required libraries in your Jupyter server. Run once.

In [1]:
import sys
!{sys.executable} -m pip install pyserial
!{sys.executable} -m pip install pyvisa-py
!{sys.executable} -m pip install pyvisa

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple
Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple


Import the library and select the python visa backend

The command list_resources() shows a list of connected instruments. you can use the different ports to be used in the open_resource() call of the next cell.

In [87]:
import pyvisa
import time
from datetime import datetime

rm = pyvisa.ResourceManager('@py')
rm.list_resources()

('ASRL/dev/ttyUSB4::INSTR',
 'ASRL/dev/ttyUSB3::INSTR',
 'ASRL/dev/ttyUSB2::INSTR',
 'ASRL/dev/ttyUSB1::INSTR',
 'ASRL/dev/ttyUSB0::INSTR',
 'ASRL/dev/ttyAMA0::INSTR')

This cell open the specified instrument and shows its identification. Use the identification to check that you are speaking with the correct instrument.

In [56]:
dmm = rm.open_resource('ASRL/dev/ttyUSB1')
#dmm.write("*RST")
#time.sleep(10)
print(dmm.query("*IDN?")) 

Keysight Technologies,U1242C,MY59190061,V1.22



In [88]:
for _ in range(50): 
    measure = dmm.query("READ?")
    timestampStr = datetime.now().strftime("%H:%M:%S.%f")
    print(timestampStr, " Measure: ", measure)

21:50:42.596828  Measure:  +1.78000000E+01

21:50:42.628386  Measure:  +1.78000000E+01

21:50:42.659613  Measure:  +1.78000000E+01

21:50:42.696763  Measure:  +1.78000000E+01

21:50:42.726925  Measure:  +1.78000000E+01

21:50:42.757132  Measure:  +1.78000000E+01

21:50:42.803565  Measure:  +1.78000000E+01

21:50:42.834016  Measure:  +1.78000000E+01

21:50:42.864366  Measure:  +1.78000000E+01

21:50:42.898495  Measure:  +1.78000000E+01

21:50:42.928735  Measure:  +1.78000000E+01

21:50:42.960079  Measure:  +1.78000000E+01

21:50:43.001836  Measure:  +1.77900000E+01

21:50:43.032013  Measure:  +1.77900000E+01

21:50:43.062275  Measure:  +1.77900000E+01

21:50:43.100138  Measure:  +1.77900000E+01

21:50:43.132562  Measure:  +1.77900000E+01

21:50:43.164921  Measure:  +1.77900000E+01

21:50:43.205167  Measure:  +1.77800000E+01

21:50:43.235370  Measure:  +1.77800000E+01

21:50:43.265565  Measure:  +1.77800000E+01

21:50:43.305100  Measure:  +1.77800000E+01

21:50:43.335318  Measure:  +1.77